# Quail segmentation and feature extraction exploration #

*11 January 2023*

Based on `Segmentation_01.ipyb` from co-coding with Robert Haase last week.

<!-- I'd like the full workflow in Jupyter; then add exploration with other background de-noising or sklearn Random Forest Segmentation. -->

Robert Haase advises small, modular notebooks. Splitting `Segmentation_02_exporation_m1.ipynb` into smaller units.

## Import tools ##

In [1]:
import napari
import napari_segment_blobs_and_things_with_membranes as nsbatwm  # version 0.3.3
import napari_crop
import pyclesperanto_prototype as cle  # version 0.21.2
import pandas as pd

from napari_simpleitk_image_processing import label_statistics
from skimage.io import imread, imsave

import napari_skimage_regionprops as nsr

from sklearn.preprocessing import StandardScaler
import seaborn
import stackview

## Set filenames ##

In [2]:
source_stack          = '/Volumes/Quail/Quail/220214_highresnode/220214_63x_node_scan_121t_11x11y_MERGED_rotated.tif'
datafolder            = 'data/'
cropped_node_file     = f'{datafolder}220214_cropped_node.tif'
cropped_non_node_file = f'{datafolder}220214_cropped_non_node.tif'

## Open viewer ##

In [3]:
viewer = napari.Viewer()

## Add source stack ##

In [4]:
# viewer.open('/Volumes/Quail/Quail/220214_highresnode/220214_63x_node_scan_121t_11x11y_MERGED_rotated.tif')
highres_node = imread(source_stack)

In [5]:
viewer.add_image(
    highres_node,
    colormap='gray_r'
)

<Image layer 'highres_node' at 0x15cd04220>

## Define crop regions ##

One in Henson's node, the other in the periphery of the tissue.

In [6]:
node_crop_region = [
    [100, 6359.53378526, 6166.86457369],
    [100, 6359.53378526, 8045.70961167],
    [100, 8238.37882323, 8045.70961167],
    [100, 8238.37882323, 6166.86457369]
]

In [7]:
non_node_crop_region = [
    [   67.        ,  6606.34456304, 10653.08839096],
    [   67.        ,  6606.34456304, 12536.25288005],
    [   67.        ,  8489.50905213, 12536.25288005],
    [   67.        ,  8489.50905213, 10653.08839096]
]

In [8]:
viewer.add_shapes(
    node_crop_region,
    name='node_crop_region',
    opacity=0.5,
    ndim=3,
)

<Shapes layer 'node_crop_region' at 0x15cf1db40>

In [9]:
viewer.add_shapes(
    non_node_crop_region,
    name='non_node_crop_region',
    opacity=0.5,
    ndim=3,
)

<Shapes layer 'non_node_crop_region' at 0x15d97cd60>

In [10]:
viewer.layers

[<Image layer 'highres_node' at 0x15cd04220>, <Shapes layer 'node_crop_region' at 0x15cf1db40>, <Shapes layer 'non_node_crop_region' at 0x15d97cd60>]

## Crop ##

Uses napari_crop plugin.

In [11]:
cropped_node = napari_crop.crop_region(layer=viewer.layers[0], shapes_layer=viewer.layers[1])[0][0] # cropped node returnds a list of [(data, metadata_dictionary)] tuples for each shape
cropped_node.shape

(201, 1879, 1880)

In [12]:
cropped_non_node = napari_crop.crop_region(layer=viewer.layers[0], shapes_layer=viewer.layers[2])[0][0] # cropped node returnds a list of [(data, metadata_dictionary)] tuples for each shape
cropped_non_node.shape

(201, 1885, 1884)

## View cropped regions ##

In [13]:
viewer.add_image(
    cropped_node,
    colormap='gray_r'
)

<Image layer 'cropped_node' at 0x15da430a0>

In [14]:
viewer.add_image(
    cropped_non_node,
    colormap='gray_r'
)

<Image layer 'cropped_non_node' at 0x15da1c310>

## Save cropped regions ##

In [15]:
imsave(cropped_node_file, cropped_node, plugin='tifffile')

In [16]:
imsave(cropped_non_node_file, cropped_non_node, plugin='tifffile')

## Close unneeded layers ##

Free up memory.

In [17]:
viewer.layers.remove('highres_node')

In [18]:
viewer.layers.remove('node_crop_region')

In [19]:
viewer.layers.remove('non_node_crop_region')

In [20]:
# viewer.layers.remove('cropped_node')

In [21]:
# viewer.layers.remove('cropped_non_node')

## End ##